# Comparing log-determinants and tracing in tensorflow

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# fix generator for random numbers
rng = np.random.default_rng(seed=145452452)

In [3]:
def logdet(tensor):
    """Compute the log-determinant of a tensor
    
    Parameters
    ----------
    tensor : tf.Tensor
        Should have shape [k, n, n]
    
    Returns
    logdet : tf.double 
        Log-determinant summed over 0th axis.
    """
    return tf.math.reduce_sum(tf.linalg.logdet(tensor), axis=0)

In [4]:
# sample psd symmetrtic matrices
N = 50
Bs = [rng.uniform(size=(100, 10)) for _ in range(N)]
As = [B @ B.T  + 0.1 + np.eye(100) for B in Bs]
arr = np.stack(As, axis=0)
print(arr.shape)
mat = tf.constant(arr, dtype=tf.double)
print(mat.shape)

(50, 100, 100)
(50, 100, 100)


In [5]:
%%timeit
logdet(mat)

691 µs ± 7.72 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [6]:
@tf.function()
def logdet(tensor):
    """Compute the log-determinant of a tensor
    
    Parameters
    ----------
    tensor : tf.Tensor
        Should have shape [k, n, n]
    
    Returns
    logdet : tf.double 
        Log-determinant summed over 0th axis.
    """
    return tf.math.reduce_sum(tf.linalg.logdet(tensor), axis=0)

In [7]:
# run once to engage tracing
val = logdet(arr)
print(f"{val:.2f}")

1277.53


2022-12-23 17:11:03.022629: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [8]:
%%timeit
logdet(mat)

773 µs ± 77 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
